In [23]:
import numpy as np
import pandas as pd
import random
import torch

In [2]:
dataDir = '/usr/sci/cibc/Maprodxn/ClinicalECGData/AllClinicalECGs/'

timeCutoff = 900 #seconds
lowerCutoff = 0 #seconds

kclCohort = np.load(dataDir+'kclCohort_v1.npy',allow_pickle=True)
data_types = {
    'DeltaTime': float,   
    'KCLVal': float,    
    'ECGFile': str,     
    'PatId': int,       
    'KCLTest': str      
}
kclCohort = pd.DataFrame(kclCohort,columns=['DeltaTime','KCLVal','ECGFile','PatId','KCLTest']) 
for key in data_types.keys():
	kclCohort[key] = kclCohort[key].astype(data_types[key])

print(kclCohort.shape)
kclCohort = kclCohort[kclCohort['DeltaTime']<=timeCutoff]
kclCohort = kclCohort[kclCohort['DeltaTime']>lowerCutoff]
print(kclCohort.shape)


(42440729, 5)
(169128, 5)


In [3]:
kclCohort = kclCohort.dropna(subset=['DeltaTime']) 
kclCohort = kclCohort.dropna(subset=['KCLVal']) 
print(kclCohort.shape)


(168660, 5)


In [4]:
ix = kclCohort.groupby('ECGFile')['DeltaTime'].idxmin()
kclCohort = kclCohort.loc[ix]
print(kclCohort.shape)


(164608, 5)


In [5]:
numECGs = len(kclCohort)
numPatients = len(np.unique(kclCohort['PatId']))
numECGs, numPatients

(164608, 106754)

In [6]:
print('setting up train/val split')
numTest = int(0.1 * numPatients)
numTrain = numPatients - numTest
assert (numPatients == numTrain + numTest), "Train/Test spilt incorrectly"
RandomSeedSoAlswaysGetSameDatabseSplit = 1
patientIds = list(np.unique(kclCohort['PatId']))
random.Random(RandomSeedSoAlswaysGetSameDatabseSplit).shuffle(patientIds)

trainPatientInds = patientIds[:numTrain]
testPatientInds = patientIds[numTrain:numTest + numTrain]
trainECGs = kclCohort[kclCohort['PatId'].isin(trainPatientInds)]
testECGs = kclCohort[kclCohort['PatId'].isin(testPatientInds)]
print(trainECGs.shape, testECGs.shape)

setting up train/val split
(148157, 5) (16451, 5)


In [8]:
kclTaskParams = dict(highThresh = 5, lowThresh=4)
trainECGs_normal = trainECGs[(trainECGs['KCLVal']>=kclTaskParams['lowThresh']) & (trainECGs['KCLVal']<=kclTaskParams['highThresh'])]
trainECGs_abnormal = trainECGs[ (trainECGs['KCLVal']>kclTaskParams['highThresh'])]
print( trainECGs_normal.shape, trainECGs_abnormal.shape)

testECGs_normal = testECGs[(testECGs['KCLVal']>=kclTaskParams['lowThresh']) & (testECGs['KCLVal']<=kclTaskParams['highThresh'])]
testECGs_abnormal = testECGs[ (testECGs['KCLVal']>kclTaskParams['highThresh'])]
print(testECGs_normal.shape, testECGs_abnormal.shape)

(70598, 5) (6586, 5)
(7835, 5) (712, 5)


In [9]:
import DataTools as DD
trainECG_normal_dataset = DD.ECG_KCL_Datasetloader(
    baseDir= dataDir + 'pythonData/',
    ecgs=trainECGs_normal['ECGFile'].tolist(),
    kclVals=trainECGs_normal['KCLVal'].tolist(),
    normalize=False,
    allowMismatchTime=False,
    randomCrop=True,
)

trainECG_abnormal_dataset = DD.ECG_KCL_Datasetloader(
    baseDir= dataDir + 'pythonData/',
    ecgs=trainECGs_abnormal['ECGFile'].tolist(),
    kclVals=trainECGs_abnormal['KCLVal'].tolist(),
    normalize=False,
    allowMismatchTime=False,
    randomCrop=True,
    
)


testECG_normal_dataset = DD.ECG_KCL_Datasetloader(
    baseDir= dataDir + 'pythonData/',
    ecgs=testECGs_normal['ECGFile'].tolist(),
    kclVals=testECGs_normal['KCLVal'].tolist(),
    normalize=False,
    allowMismatchTime=False,
    randomCrop=True,
)

testECG_abnormal_dataset = DD.ECG_KCL_Datasetloader(
    baseDir= dataDir + 'pythonData/',
    ecgs=testECGs_abnormal['ECGFile'].tolist(),
    kclVals=testECGs_abnormal['KCLVal'].tolist(),
    normalize=False,
    allowMismatchTime=False,
    randomCrop=True,
    
)

In [10]:
ecg,kcl = trainECG_normal_dataset[0]
ecg, kcl = trainECG_abnormal_dataset[0]
ecg, kcl = testECG_normal_dataset[0]
ecg, kcl = testECG_abnormal_dataset[0]
print(len(trainECG_normal_dataset), len(trainECG_abnormal_dataset), len(testECG_normal_dataset), len(testECG_abnormal_dataset))

70598 6586 7835 712


In [24]:
# Create dataloader of the dataset
trainECG_abnormal_dataloader = torch.utils.data.DataLoader(trainECG_abnormal_dataset, batch_size=32, shuffle=True)

In [36]:
ecg, kcls = next(iter(trainECG_abnormal_dataloader))

In [ ]:
# get max value's index of kcls
torch.argmax(kcls)


tensor([5.7000, 5.8000, 5.3000, 5.7000, 7.8000, 5.3000, 5.5000, 6.8000, 5.9000,
        6.6000, 5.2000, 6.4000, 5.1000, 6.0000, 5.1000, 5.6000, 6.1000, 5.2000,
        5.5000, 5.5000, 6.3000, 5.2000, 7.8000, 5.2000, 5.9000, 5.5000, 7.2000,
        6.5000, 5.2000, 6.8000, 5.8000, 5.5000])

In [ ]:
import subprocess
import os
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

def check_gpu_availability(min_free_memory_mb=35000):
    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.free', '--format=csv,noheader,nounits'], capture_output= True, text=True)
    free_memory_list = [int(x) for x in result.stdout.strip().split('\n')]

    free_gpus = []
    for idx, free_memory in enumerate(free_memory_list):
        if free_memory >= min_free_memory_mb:
            free_gpus.append(idx)
            
    return free_gpus

def run_process_on_gpu(gpu_indices, process_command):
    
    if(len(gpu_indices) == 0):
        print("No available GPUs found.")
        return
    
    devices = ""
    for idx in gpu_indices:
        devices += f"{idx},"
    devices = devices[:-1]
    
    os.environ["CUDA_VISIBLE_DEVICES"] = devices
    
    process = subprocess.Popen(process_command, shell=True)
    
    #SENDING AN EMAIL
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    username = 'deekshithsender@gmail.com'
    password = 'bnvf hzzb hbxw vgki'
    from_addr = 'deekshithsender@gmail.com'
    to_addr = 'deekshithreceiver@gmail.com'

    # Email content
    subject = 'System Information'
    body = f'Process started on GPU(s): {devices} @ {time.ctime()}'
    
    html_body = f"<pre>{body}</pre>"
    msg.attach(MIMEText(html_body, 'html'))
    
    msg = MIMEMultipart()
    msg['Subject'] = subject
    msg['From'] = from_addr
    msg['To'] = to_addr
    
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(username, password)
        server.sendmail(from_addr, to_addr, msg.as_string())
        print(f"Sent an EMAIL at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")
    
    process.wait()
    print(f"Porcess finished on GPU(s): {devices}")
    
def main():
    
    process_command = "python kclExplainabiity.py"
    
    while True:
        gpu_indices = check_gpu_availability(min_free_memory_mb=37000)
        
        if len(gpu_indices) > 0:
            run_process_on_gpu(gpu_indices, process_command)
            break
        else:
            print("Waiting for available GPU...")
            time.sleep(20)



1,2,3
